<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#Linear-regression" data-toc-modified-id="Linear-regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Linear regression</a></span><ul class="toc-item"><li><span><a href="#The-normal-equation" data-toc-modified-id="The-normal-equation-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>The normal equation</a></span></li><li><span><a href="#Batch-gradient-descent-(BGD)" data-toc-modified-id="Batch-gradient-descent-(BGD)-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Batch gradient descent (BGD)</a></span></li><li><span><a href="#Stochastic-gradient-descent-(SGD)" data-toc-modified-id="Stochastic-gradient-descent-(SGD)-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Stochastic gradient descent (SGD)</a></span></li><li><span><a href="#Mini-batch-gradient-descent-(MBGD)" data-toc-modified-id="Mini-batch-gradient-descent-(MBGD)-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Mini-batch gradient descent (MBGD)</a></span></li></ul></li></ul></li></ul></div>

<b>
<p>
<center>
<font size="5">
Popular Machine Learning Methods: Idea, Practice and Math
</font>
</center>
</p>
    
<p>
<center>
<font size="4">
Models: Shallow Learning
</font>
</center>
</p>

<p>
<center>
<font size="3">
Data Science, Columbian College of Arts & Sciences, George Washington University
</font>
</center>
</p>

<p>
<center>
<font size="3">
Yuxiao Huang
</font>
</center>
</p>
</b>

# Overview

- This notebook includes some common models used in PMLM.
- Concretely, these models are:
    - linear regression
    - logistic regression
    - single / multiple layer perceptron 
- See the accompanied slides in our [github repository](https://github.com/yuxiaohuang/teaching/tree/master/gwu/machine_learning_I/fall_2020/slides).

# Models

## Linear regression

### The normal equation

The code below shows how to implement linear regression using the normal equation.

In [1]:
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np

class LinearRegression_NE(BaseEstimator, RegressorMixin):
    """Linear regression implemented using the normal equation"""

    def fit(self, X, y):
        """
        The fit function
        
        Parameters
        ----------
        X : the feature matrix
        y : the target vector
        """
        
        # Get the augmented feature matrix, [1, X]
        IX = np.hstack((np.ones((X.shape[0], 1)), X))

        # Get the optimal solution using the normal equation
        self.theta = np.linalg.pinv(IX).dot(y)
        
        # Get the predicted target vector
        y_pred = self.net_input(IX)
                        
        # Get the loss (MSE)
        self.loss = ((y - y_pred) ** 2).sum() / IX.shape[0]
        
    def net_input(self, IX):
        """
        Get the predicted target vector
        
        Parameters
        ----------
        IX : The augmented feature matrix [1, X]
        
        Returns
        ----------
        The predicted target vector
        """
        
        return IX.dot(self.theta)

    def predict(self, X):
        """
        The predict function
        
        Parameters
        ----------
        X : the feature matrix
        
        Returns
        ----------
        The predicted value of the target
        """
        
        # The augmented feature matrix [1, X]
        IX = np.hstack((np.ones((X.shape[0], 1)), X))
        
        return self.net_input(IX)

### Batch gradient descent (BGD)

The code below shows how to implement linear regression using batch gradient descent and regularization (lasso, ridge and elastic net).

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np

class LinearRegression_BGD(BaseEstimator, RegressorMixin):
    """Linear regression implemented using batch gradient descent and regularization (lasso, ridge and elastic net)"""
    
    def __init__(self, 
                 max_iter=100, 
                 eta=10 ** -2,
                 penalty='l2',
                 alpha=0.0001, 
                 gamma=0.15,
                 random_state=42):
        
        # The maximum number of epochs
        self.max_iter = max_iter
        
        # The learning rate
        self.eta = eta
        
        # The regularization term
        self.penalty=penalty
        
        # The regularization parameter
        self.alpha=alpha

        # The elastic net mixing parameter
        self.gamma=gamma

        # The random state
        self.random_state = random_state

    def fit(self, X_train, y_train, X_val=None, y_val=None):
        """
        The fit function
        
        Parameters
        ----------
        X_train : The training feature matrix
        y_train : The training target vector
        X_val : The validation feature matrix
        y_val : The validation target vector
        """
        
        # Get the augmented training feature matrix, [1, X_train]
        IX_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        
        # Get the random number generator
        self.rgen = np.random.RandomState(seed=self.random_state)
        
        # Initialize the parameters
        self.theta = self.rgen.normal(loc=0.0, scale=0.01, size=IX_train.shape[1])
        
        # Initialize the training and validation loss
        self.loss_train, self.loss_val = [], []
        
        # For each epoch
        for _ in range(self.max_iter):
            # Get the predicted target vector on the training data
            y_train_pred = self.net_input(IX_train)
            
            # Get the training error
            error_train = y_train - y_train_pred
                        
            # Get the training mse
            mse_train = (error_train ** 2).sum() / IX_train.shape[0]
            
            # Update the parameters
            # If no regularization
            if self.penalty == None:
                self.theta += self.eta * (2 / IX_train.shape[0] * IX_train.T.dot(error_train))
            # If lasso
            elif self.penalty == 'l1':
                self.theta += self.eta * (2 / IX_train.shape[0] * IX_train.T.dot(error_train) 
                                          - self.alpha * np.append([0], np.sign(self.theta[1:])))
            # If ridge
            elif self.penalty == 'l2':
                self.theta += self.eta * (2 / IX_train.shape[0] * IX_train.T.dot(error_train) 
                                          - self.alpha * np.append([0], self.theta[1:]))
            # If elastic net
            elif self.penalty == 'elasticnet':
                self.theta += self.eta * (2 / IX_train.shape[0] * IX_train.T.dot(error_train) 
                                          - self.alpha * self.gamma * np.append([0], np.sign(self.theta[1:])) 
                                          - self.alpha * (1 - self.gamma) * np.append([0], self.theta[1:]))
                           
            # Update the training loss
            self.loss_train.append(mse_train)
            
            # If the validation feature matrix and target vector are available
            if X_val is not None and y_val is not None:
                # Transform X_train and y_train to numpy array
                X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
        
                # Get the augmented validation feature matrix, [1, X_val]
                IX_val = np.hstack((np.ones((X_val.shape[0], 1)), X_val))
                
                # Get the predicted target vector on the validation data
                y_val_pred = self.net_input(IX_val)
                
                # Get the validation error
                error_val = y_val - y_val_pred
                
                # Get the validation mse
                mse_val = (error_val ** 2).sum() / IX_val.shape[0]
                
                # Update the validation loss
                self.loss_val.append(mse_val)

    def net_input(self, IX):
        """
        Get the predicted target vector
        
        Parameters
        ----------
        IX : The augmented feature matrix [1, X]
        
        Returns
        ----------
        The predicted target vector
        """
        
        return IX.dot(self.theta)

    def predict(self, X):
        """
        The predict function
        
        Parameters
        ----------
        X : the feature matrix
        
        Returns
        ----------
        The predicted value of the target
        """
        
        # The augmented feature matrix [1, X]
        IX = np.hstack((np.ones((X.shape[0], 1)), X))
        
        return self.net_input(IX)

### Stochastic gradient descent (SGD)

The code below shows how to implement linear regression using stochastic gradient descent and regularization (lasso, ridge and elastic net).

In [3]:
class LinearRegression_SGD(BaseEstimator, RegressorMixin):
    """Linear regression implemented using stochastic gradient descent and regularization (lasso, ridge and elastic net)"""
    
    def __init__(self, 
                 max_iter=100,
                 shuffle=True,
                 eta=10 ** -2, 
                 penalty='l2',
                 alpha=0.0001, 
                 gamma=0.15,
                 random_state=42):
        
        # The maximum number of epochs
        self.max_iter = max_iter
        
        # Whether to shuffle samples in each epoch
        self.shuffle = shuffle
        
        # The learning rate
        self.eta = eta
        
        # The regularization term
        self.penalty=penalty
        
        # The regularization parameter
        self.alpha=alpha

        # The elastic net mixing parameter
        self.gamma=gamma
        
        # The random state
        self.random_state = random_state

    def fit(self, X_train, y_train, X_val=None, y_val=None):
        """
        The fit function
        
        Parameters
        ----------
        X_train : The training feature matrix
        y_train : The training target vector
        X_val : The validation feature matrix
        y_val : The validation target vector
        """
        
        # Get the augmented training feature matrix, [1, X_train]
        IX_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        
        # Get the indices of the augmented training feature matrix
        idxs_train = np.array(range(IX_train.shape[0]))
        
        # Get the random number generator
        self.rgen = np.random.RandomState(seed=self.random_state)
        
        # Initialize the parameters
        self.theta = self.rgen.normal(loc=0.0, scale=0.01, size=IX_train.shape[1])
        
        # Initialize the training and validation loss
        self.loss_train, self.loss_val = [], []
        
        # For each epoch
        for _ in range(self.max_iter):
            if self.shuffle is True:
                # Shuffle the indices
                self.rgen.shuffle(idxs_train)
                
            # Initialize the mse
            mse_train = 0
            
            # For each sample
            for i in idxs_train:                
                # Get the predicted target vector on the training data
                y_train_pred = self.net_input(IX_train[i, :])

                # Get the training error
                error_train = y_train[i] - y_train_pred

                # Get the training mse
                mse_train += (error_train ** 2) / IX_train.shape[0]

                # Update the parameters
                # If no regularization
                if self.penalty == None:
                    self.theta += self.eta * (2 * IX_train[i, :].T.dot(error_train))
                # If lasso
                elif self.penalty == 'l1':
                    self.theta += self.eta * (2 * IX_train[i, :].T.dot(error_train) 
                                              - self.alpha * np.append([0], np.sign(self.theta[1:])))
                # If ridge
                elif self.penalty == 'l2':
                    self.theta += self.eta * (2 * IX_train[i, :].T.dot(error_train) 
                                              - self.alpha * np.append([0], self.theta[1:]))
                # If elastic net
                elif self.penalty == 'elasticnet':
                    self.theta += self.eta * (2 * IX_train[i, :].T.dot(error_train) 
                                              - self.alpha * self.gamma * np.append([0], np.sign(self.theta[1:])) 
                                              - self.alpha * (1 - self.gamma) * np.append([0], self.theta[1:]))

            # Update the training loss
            self.loss_train.append(mse_train)

            # If the validation feature matrix and target vector are available
            if X_val is not None and y_val is not None:
                # Get the augmented validation feature matrix, [1, X_val]
                IX_val = np.hstack((np.ones((X_val.shape[0], 1)), X_val))

                # Get the predicted target vector on the validation data
                y_val_pred = self.net_input(IX_val)

                # Get the validation error
                error_val = y_val - y_val_pred

                # Get the validation mse
                mse_val = (error_val ** 2).sum() / IX_val.shape[0]

                # Update the validation loss
                self.loss_val.append(mse_val)
                
    def net_input(self, IX):
        """
        Get the predicted target vector
        
        Parameters
        ----------
        IX : The augmented feature matrix [1, X]
        
        Returns
        ----------
        The predicted target vector
        """
        
        return IX.dot(self.theta)

    def predict(self, X):
        """
        The predict function
        
        Parameters
        ----------
        X : the feature matrix
        
        Returns
        ----------
        The predicted value of the target
        """
        
        # The augmented feature matrix [1, X]
        IX = np.hstack((np.ones((X.shape[0], 1)), X))
        
        return self.net_input(IX)

### Mini-batch gradient descent (MBGD)

The code below shows how to implement linear regression using mini-batch gradient descent and regularization (lasso, ridge and elastic net).

In [4]:
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np

class LinearRegression_MBGD(BaseEstimator, RegressorMixin):
    """Linear regression implemented using mini-batch gradient descent and regularization (lasso, ridge and elastic net)"""
    
    def __init__(self,
                 max_iter=100,
                 shuffle=True,
                 batch_size=32,
                 eta=10 ** -2, 
                 penalty='l2',
                 alpha=0.0001, 
                 gamma=0.15,
                 random_state=42):
        
        # The maximum number of epochs
        self.max_iter = max_iter
        
        # Whether to shuffle samples in each epoch
        self.shuffle = shuffle
        
        # The size of minibatches for stochastic optimizers
        self.batch_size = batch_size
        
        # The learning rate
        self.eta = eta
        
        # The regularization term
        self.penalty=penalty
        
        # The regularization parameter
        self.alpha=alpha

        # The elastic net mixing parameter
        self.gamma=gamma
        
        # The random state
        self.random_state = random_state

    def fit(self, X_train, y_train, X_val=None, y_val=None):
        """
        The fit function
        
        Parameters
        ----------
        X_train : The training feature matrix
        y_train : The training target vector
        X_val : The validation feature matrix
        y_val : The validation target vector
        """
        
        # Get the augmented training feature matrix, [1, X_train]
        IX_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        
        # Get the random number generator
        self.rgen = np.random.RandomState(seed=self.random_state)
        
        # Initialize the parameters
        self.theta = self.rgen.normal(loc=0.0, scale=0.01, size=IX_train.shape[1])
        
        # Initialize the training and validation loss
        self.loss_train, self.loss_val = [], []
        
        # For each epoch
        for _ in range(self.max_iter):
            # Get the indices of the training data
            idxs_train = np.array(range(IX_train.shape[0]))
            
            # Get the minibatches of the training data
            mbs = self.get_minibatches(idxs_train)

            # Initialize the training mse
            mse_train = 0

            # For each minibatch
            for mb in mbs:   
                # Get the augmented training feature matrix and target vector
                IX_train_mb, y_train_mb = IX_train[mb,:], y_train[mb]

                # Get the predicted target vector on the training data
                y_train_mb_pred = self.net_input(IX_train_mb)

                # Get the training error
                error_train = y_train_mb - y_train_mb_pred

                # Get the training mse
                mse_train += (error_train ** 2).sum() / IX_train.shape[0]
                
                # Update the parameters
                # If no regularization
                if self.penalty == None:
                    self.theta += self.eta * (2 / IX_train_mb.shape[0] * IX_train_mb.T.dot(y_train_mb - y_train_mb_pred))
                # If lasso
                elif self.penalty == 'l1':
                    self.theta += self.eta * (2 / IX_train_mb.shape[0] * IX_train_mb.T.dot(y_train_mb - y_train_mb_pred) 
                                              - self.alpha * np.append([0], np.sign(self.theta[1:])))
                # If ridge
                elif self.penalty == 'l2':
                    self.theta += self.eta * (2 / IX_train_mb.shape[0] * IX_train_mb.T.dot(y_train_mb - y_train_mb_pred) 
                                              - self.alpha * np.append([0], self.theta[1:]))
                # If elastic net
                elif self.penalty == 'elasticnet':
                    self.theta += self.eta * (2 / IX_train_mb.shape[0] * IX_train_mb.T.dot(y_train_mb - y_train_mb_pred) 
                                              - self.alpha * self.gamma * np.append([0], np.sign(self.theta[1:])) 
                                              - self.alpha * (1 - self.gamma) * np.append([0], self.theta[1:]))

            # Update the training loss
            self.loss_train.append(mse_train)
            
            # If the validation feature matrix and target vector are available
            if X_val is not None and y_val is not None:
                # Get the augmented validation feature matrix, [1, X_val]
                IX_val = np.hstack((np.ones((X_val.shape[0], 1)), X_val))
                
                # Get the predicted target vector on the validation data
                y_val_pred = self.net_input(IX_val)
                
                # Get the validation error
                error_val = y_val - y_val_pred
                
                # Get the validation mse
                mse_val = (error_val ** 2).sum() / IX_val.shape[0]
                
                # Update the validation loss
                self.loss_val.append(mse_val)

    def get_minibatches(self, idxs):
        """
        Get the minibatches
        
        Parameters
        ----------
        idxs : The indices of the data
        
        Returns
        ----------
        The minibatches
        """
        
        # Initialize the minibatches
        mbs = []
        
        if self.shuffle is True:
            # Shuffle the indices
            self.rgen.shuffle(idxs)
                
        # Get the number of minibatches
        n_batch = len(idxs) // self.batch_size
        
        # For each minibatch
        for i in range(n_batch):
            # Get the first and last index (exclusive) of the minibatch
            first_idx = i * self.batch_size
            last_idx = min((i + 1) * self.batch_size, len(idxs))
                                    
            # Get the minibatch
            mb = idxs[first_idx : last_idx]
            
            # Update the minibatches
            mbs.append(mb)

        return mbs

    def net_input(self, IX):
        """
        Get the predicted target vector
        
        Parameters
        ----------
        IX : The augmented feature matrix [1, X]
        
        Returns
        ----------
        The predicted target vector
        """
        
        return IX.dot(self.theta)

    def predict(self, X):
        """
        The predict function
        
        Parameters
        ----------
        X : the feature matrix
        
        Returns
        ----------
        The predicted value of the target
        """
        
        # The augmented feature matrix [1, X]
        IX = np.hstack((np.ones((X.shape[0], 1)), X))
        
        return self.net_input(IX)